In [1]:
import re
import requests
from urllib.request import quote
from collections import defaultdict

from IPython.display import HTML, display

In [2]:
APP_CONFIG = [
    #CCFS
    #('iPhone', '2328'),('iPad', '2329'),('Android','2330') 
    #FHS
    #('iPhone', '708'),('iPad', '797'),('Android','627') 
    #PRS  
    #('Android', '608'),('iPad', '796'),('iPhone','607') 
    #BW3S
    ('Android', '1063'),('iPad', '1061'),('iPhone','1060') 
    #CCJS
    #('Android', '1412'),('iPad', '1411'),('iPhone','1410') 
]
EXCHANGES = [
    'mopub', 'mopub2', 'nexage', 'adx', 'rubicon', 'omax', 'applovin', 'chartboost', 'appnexus', 'unity',
    'rubicon_west', 'adx_west', 'applovin_west', 'appnexus_west', 'inmobi_west', 'mobilityware_west',
    'adx_asia', 'applovin_asia', 'rubicon_asia', 'inmobi_asia',
    'adx_eu', 'applovin_eu', 'inmobi_eu'
]
GRP_TARGET = 'sortByMaxima(summarize(groupByNode(rtb.campaign.{}.app{}.*.filtered_out.*,6,"sumSeries"),"{}","sum",true))'
GRP_URL = 'http://192.168.{}.91/render?target={}&from=-1d&format=json'
GRP_RENDER_URL = 'http://192.168.{}.91/render?target={}&from=-1d&height=800&width=1200&lineWidth=2&title=Filtered+out+reasons&vtitle=QPH&hideLegend=0'
TARGET_RE = re.compile('summarize\(([^,]+),')
EXCHANGE_DC_IP = {'west': 19, 'asia': 20, 'eu': 21}

In [3]:
data = []
try:
    for device, app_id in APP_CONFIG:
        print('trying on '+ device)
        config_data = []
        data.append((device, config_data))
        for exchange in EXCHANGES:
            print ('trying on '+ exchange)
            values = []
            dc_ip = EXCHANGE_DC_IP.get(exchange.split('_')[-1], 18)
            url = GRP_URL.format(dc_ip, GRP_TARGET.format(exchange, app_id, '1d'))
            for point in requests.get(url).json():
                if point['datapoints'][0][0]:
                    values.append(TARGET_RE.search(point['target']).group(1))
            if values:
                render_url = GRP_RENDER_URL.format(dc_ip, quote(GRP_TARGET.format(exchange, app_id, '1h')))
                config_data.append((exchange, render_url, values))
            print ('success on '+ exchange+'\n'),
        print ('success on '+ device+'\n')
        
except:
    print ('something went wrong in ' + device + ', '+ exchange)
else:
    print ('success')

trying on Android
trying on mopub
success on mopub

trying on mopub2
success on mopub2

trying on nexage
success on nexage

trying on adx
success on adx

trying on rubicon
success on rubicon

trying on omax
success on omax

trying on applovin
success on applovin

trying on chartboost
success on chartboost

trying on appnexus
success on appnexus

trying on unity
success on unity

trying on rubicon_west
success on rubicon_west

trying on adx_west
success on adx_west

trying on applovin_west
success on applovin_west

trying on appnexus_west
success on appnexus_west

trying on inmobi_west
success on inmobi_west

trying on mobilityware_west
success on mobilityware_west

trying on adx_asia
success on adx_asia

trying on applovin_asia
success on applovin_asia

trying on rubicon_asia
success on rubicon_asia

trying on inmobi_asia
success on inmobi_asia

trying on adx_eu
success on adx_eu

trying on applovin_eu
success on applovin_eu

trying on inmobi_eu
success on inmobi_eu

success on Android

In [4]:
def anchor(name, url):
    return '<a href="{}">{}</a>'.format(url, name)


def td(val, rs=None, cs=None, th=False):
    return '<t{h} style="text-align:center;border:1px solid black;" {rs} {cs}>{val}</t{h}>'.format(
        val=val, rs=('rowspan="{}"'.format(rs) if rs else ''), cs=('colspan="{}"'.format(cs) if cs else ''),
        h=('h' if th else 'd'))


def tr(val):
    return '<tr>{}</tr>'.format(val)


num_reasons = min(max(len(x[2]) for _, v in data for x in v), 18)
markup = '''
<table style="table-layout:fixed;border-collapse:collapse;">
    <tr>{}{}{}</tr>
    <tr>{}</tr>
    {{}}
</table>
'''.format(td('Device', rs=2, th=True), td('Exchange', rs=2, th=True),
           td('Filtered out reasons', cs=num_reasons, th=True),
           ''.join((td(i, th=True) for i in range(1, num_reasons + 1))))

rows = []
for k, v in data:
    cells = [td(k, rs=len(v))]
    for exchange, url, reasons in v:
        cells.append(td(anchor(exchange, url)))
        cells.extend(map(td, reasons))
        cells.extend((td('') for i in range(len(reasons), num_reasons)))
        rows.append(tr(''.join(cells)))
        cells = []

markup = markup.format(''.join(rows))
display(HTML(markup))